In [1]:
import numpy as np
import subprocess
import re
from urllib.request import urlopen
from pickle import dump, load
from sklearn.tree import DecisionTreeRegressor
import subprocess
from sklearn.model_selection import KFold
from tqdm import tqdm

In [2]:
points = []
with open('points_test.txt') as file_test:
    for line in tqdm(file_test):
        line_split = line.strip('\n').split(' ')
        point = [float(x.split(':')[1]) for x in line_split]
        points.append(point)
points = np.array(points)

4000000it [00:20, 199047.30it/s]


In [3]:
points = np.array(points)

In [ ]:
with open('model' + str(209958_300)  + '.pkl', 'rb') as f:
    model = load(f)

In [ ]:
y_pred = model.predict(points)

In [ ]:
y_pred.shape

In [ ]:
points_num = np.arange(points.shape[0]) + 1

In [ ]:
out_lines = ['Id,Expected\n']
for i in range(y_pred.shape[0]):
    out_lines.append(str(i+1) + ',' + str(y_pred[i]) + '\n')

In [ ]:
with open('submission_209958_300.txt', 'w') as f_out:
    f_out.writelines(out_lines)
f_out.close()

In [6]:
def submit(model, file_name):
    y_pred = model.predict(points)
    out_lines = ['Id,Expected\n']
    for i in range(y_pred.shape[0]):
        out_lines.append(str(i+1) + ',' + str(y_pred[i]) + '\n')
    with open(file_name, 'w') as f_out:
        f_out.writelines(out_lines)
    f_out.close()

In [ ]:
with open('model' + str(207282_800)  + '.pkl', 'rb') as f:
    model_800 = load(f)

In [ ]:
submit(model_800, 'submission_207282_800.txt')

In [ ]:
FILE_POINTS_PATH = 'points.txt'

def load_value(array, path=FILE_POINTS_PATH):
    array = array.reshape(-1)
    url = 'http://185.86.144.251/oracle?x1={:f}&x2={:f}&x3={:f}&x4={:f}&x5={:f}&x6={:f}&x7={:f}&x8={:f}&x9={:f}&x10={:f}'.format(
        *array)

    with urlopen(url,timeout=50) as response, open(path, 'a') as out_file:
        data = response.read().decode('utf-8')
        parsed_data = re.split(r'^Function value = |\nAttempts left = ', data)
        if (data != 'UNDIFINED'):
            result, attempts = [np.float(number)
                                for number in parsed_data if number]
            out_file.write(','.join([str(x) for x in array]) + ',' + str(result) + ',' + str(attempts) + '\n')
        else:
            out_file.write(','.join([str(x) for x in array]) + ',' + str(np.inf) + ',-1\n')
            print('inf value')
            return np.inf
    return result

In [ ]:
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from IPython.display import clear_output
from tqdm import tqdm
class ActiveLearner_My:

    def __init__(self, oracle_function, X_initial, y_initial, X_test, y_test, steps=20, dimension=10, learner_class=DecisionTreeRegressor, comittee_size=3):
        self.oracle_function = oracle_function
        self.dimension = dimension
        # number of initial points
        self.initial = X_initial.shape[0]
        # final number of steps
        self.steps = self.initial + steps
        self.learner_class = learner_class
        self.comittee_size = comittee_size
        self.vars = []
        self.vars_min = []
        self.iter_step=5

        
        
        X_to_add = np.zeros(
            shape=(steps, self.dimension)).astype(np.longdouble)
        self.X = np.vstack((X_initial,X_to_add))
        
        y_to_add = np.zeros(shape=(steps,)).astype(np.longdouble)
        self.y = np.hstack((y_initial, y_to_add))
        
        self.labeled_indices = np.hstack((np.full(y_initial.shape[0], True), np.full(steps, False)))

        self.comittee = np.array([self.learner_class(n_estimators = 150, max_depth = 4)
                                 for _ in range(self.comittee_size)])
        print(self.X.shape)
        self.X_test = X_test
        self.y_test = y_test
        
        # losses for train and validation
        self.train_rmse = []
        self.valid_rmse = []

    def generate_random_points(self, number):
        return 10 * np.random.random_sample((number, self.dimension))


    def _train_comittee(self):
        kf = KFold(n_splits=self.comittee_size, shuffle=True)
        i = 0
        X_l, y_l = self.X[self.labeled_indices], self.y[self.labeled_indices]
        for train_index, test_index in kf.split(X_l):
            self.comittee[i] = self.comittee[i].fit(
                X_l[train_index], y_l[train_index])
            i += 1

    def _choose_point_to_label(self):
        points = self.generate_random_points(1000)
        variance = np.var([self.comittee[i].predict(points)
                          for i in range(self.comittee_size)], axis=0)

        self.vars.append(max(variance))
        self.vars_min.append(min(variance))
        return points[np.argmax(variance)]

    def _label(self, point):
        
        rand_val = 0.001 * np.random.random_sample((self.iter_step, self.dimension))
        
        new_points = np.tile(point,(self.iter_step,1)) + rand_val
        print(new_points.shape)
        for i in range(self.iter_step):
            val = self.oracle_function(new_points[i])
            if val == np.inf:
                continue
            else:
                self.X[self.iteration] = new_points[i]
                self.y[self.iteration] = val
                self.labeled_indices[self.iteration] = True
                self.iteration += 1

    def add_point(self):
        self._train_comittee()
        best_point = self._choose_point_to_label()
        #print(best_point)
        self._label(best_point)

    def save_data(self):
                with open('X' + str(self.iteration) + '.pkl', 'wb') as f:
                    dump(self.X[self.labeled_indices], f)
                with open('y' + str(self.iteration) + '.pkl', 'wb') as f:
                    dump(self.y[self.labeled_indices], f)
    def test(self):
        y_pred_valid = self.final_model.predict(self.X_test)
        y_pred_train = self.final_model.predict(self.X[self.labeled_indices])
        
        valid_rmse = np.sqrt(mean_squared_error(y_pred_valid, self.y_test))
        train_rmse = np.sqrt(mean_squared_error(y_pred_train, self.y[self.labeled_indices]))
        return train_rmse, valid_rmse
        
    
    def learn(self):
        self.iteration = self.initial
        start = self.iteration
        #for i in tqdm(range(start,self.steps)):
        while self.iteration < self.steps:
            
            self.add_point()
            print(self.iteration)

            self.save_data()
            self.final_model = self.learner_class(n_estimators = 150, max_depth = 4)
            self.final_model.fit(self.X[self.labeled_indices], self.y[self.labeled_indices])
            cur_rmse_train, cur_rmse_valid = self.test()
            self.train_rmse.append(cur_rmse_train)
            self.valid_rmse.append(cur_rmse_valid)
            clear_output(True)

            plt.figure(figsize=(12, 5))
            plt.plot(self.train_rmse,  label='train')
            plt.plot(self.valid_rmse,  label='valid')
            plt.xlabel('Epochs', fontsize=16)
            plt.ylabel('Loss', fontsize=16)
            plt.legend(loc=0, fontsize=16)
            plt.show()
            if ((self.iteration + 1) % 100 == 0):
                self.save_data()
                print('train rmse={}, valid_rmse={}'.format(cur_rmse_train, cur_rmse_valid))
            #self.iteration += 1

        self.final_model = self.learner_class(n_estimators = 150, max_depth = 4)
        self.final_model.fit(
            self.X[self.labeled_indices], self.y[self.labeled_indices])
        self.save_data()
        with open('model' + str(self.iteration) + '.pkl', 'wb') as f:
            dump(self.final_model, f)

In [ ]:
with open('learner' + str(124264) + '.pkl', 'rb') as f:
    my_learner1 = load(f)

In [ ]:
submit(my_learner1.final_model, 'submission_124264.txt')

In [ ]:
with open('model' + str(209958_800)  + '.pkl', 'rb') as f:
    model_800 = load(f)

In [ ]:
submit(model_800, 'submission_209958_800.txt')

In [7]:
with open('model' + str(207282_300)  + '.pkl', 'rb') as f:
    model_800 = load(f)

In [10]:
submit(model_800, 'submission_207282_300.txt')

In [8]:
del model_800

In [10]:
with open('model' + str(211362_1200)  + '.pkl', 'rb') as f:
    model_800 = load(f)

In [11]:
submit(model_800, 'submission_211362_1200.txt')